# Natural Statistics Cross-linguistic: 

#### MLUw analysis - random sample

----

In [6]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, "data_proc")
import contingent_extraction
import warnings
warnings.filterwarnings('ignore')

In [7]:
rand_dat_inc = pd.read_csv("../data/rand_dat_inc_master.csv",index_col=0,low_memory=False)
rand_dat_inc=rand_dat_inc[rand_dat_inc["language"]!="ara"]
rand_dat_inc=rand_dat_inc[(rand_dat_inc["target_child_age"]>=5) & (rand_dat_inc["target_child_age"]<=30)]

rand_dat_inc_cg = rand_dat_inc[rand_dat_inc["caregiver"]=="caregiver"]

rand_dat_inc_cg["contingent"] = np.where(rand_dat_inc_cg["contingent"]==1, "contingent", "non-contingent")

rand_dat_inc_cg = rand_dat_inc_cg[rand_dat_inc_cg["gloss"].notna()]
rand_dat_inc_cg = rand_dat_inc_cg[rand_dat_inc_cg["gloss"]!="xxx"]
rand_dat_inc_cg = rand_dat_inc_cg[rand_dat_inc_cg["gloss"]!="yyy"]
rand_dat_inc_cg = rand_dat_inc_cg[rand_dat_inc_cg["gloss"]!="www"]

In [8]:
# add play context and year of study

play_context = pd.read_csv("../data/context_data.csv")
play_context = play_context.rename(columns={"Corpus": "corpus_name"})

# print(play_context.to_markdown())

rand_dat_inc_cg = rand_dat_inc_cg.merge(play_context,on='corpus_name')

rand_dat_inc_cg["context"] = rand_dat_inc_cg["Location"] + rand_dat_inc_cg["Activity"]

rand_dat_inc_cg["context"] = rand_dat_inc_cg["context"].replace({"HomeBook-reading":"Home: book reading",
                                                                 "HomeInterview/Unstructured":"Home: interview/unstructured",
                                                                 "HomeNaN":"Home: unreported",
                                                                 "HomeOther":"Home: other",
                                                                 "HomeUnstructured":"Home: unstructured",
                                                                 "LabOther":"Lab: other",
                                                                 "LabTabletop play":"Lab: tabletop play",
                                                                 "LabInterview/Unstructured":"Lab: interview/unstructured",
                                                                 "LabUnstructured":"Lab: unstructured",
                                                                 np.nan:"Unreported",
                                                                 "OtherUnstructured":"Other: unstructured"})

# year of study
corpora_year = pd.read_csv("../data/corpora_year.csv")
corpora_year = corpora_year.rename(columns={"Corpora": "corpus_name"})
corpora_year = corpora_year[["corpus_name", "Year collected"]]

rand_dat_inc_cg = rand_dat_inc_cg.merge(corpora_year,on='corpus_name')

In [9]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [10]:
%%R -i rand_dat_inc_cg

library("lme4")
library("knitr")
library("broom")
library("emmeans")
library("lmerTest")
library("tidyverse")

options(scipen = 999)

Loading required package: Matrix

Attaching package: ‘lmerTest’

The following object is masked from ‘package:lme4’:

    lmer

The following object is masked from ‘package:stats’:

    step

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──
✔ ggplot2 3.4.2     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.2
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.4     ✔ forcats 1.0.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand() masks Matrix::expand()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ tidyr::pack()   masks Matrix::pack()
✖ tidyr::unpack() masks Matrix::unpack()


In [11]:
%%R -o rand_dat_inc_cg

library("kableExtra")

caregiver_type <- rand_dat_inc_cg %>%
  group_by(transcript_id) %>%
  summarise(
    caregiver_type = case_when(
      all(speaker_role == "Mother") ~ "Mother only",
      all(speaker_role == "Father") ~ "Father only",
      any(speaker_role %in% c("Mother", "Father")) ~ "Mother & Father",
      TRUE ~ "Unknown"
    )
  )

rand_dat_inc_cg <- rand_dat_inc_cg %>%
  left_join(caregiver_type)

Joining with `by = join_by(transcript_id)`



Attaching package: ‘kableExtra’

The following object is masked from ‘package:dplyr’:

    group_rows



In [12]:
rand_mlu_stats = (rand_dat_inc_cg.groupby(["Language_name","target_child_id","transcript_id","contingent"])
                                  .num_tokens
                                  .agg(["mean"])
                                  .reset_index())
rand_mlu_sumstats =  rand_mlu_stats.rename({'mean': 'means'}, axis=1)

In [13]:
%%R -i rand_mlu_sumstats

# import rand_mlu_sumstats into R

NULL


In [14]:
%%R -o rand_mlu_sumstats

rand_mlu_sumstats <- rand_mlu_sumstats %>%
    left_join(caregiver_type)

Joining with `by = join_by(transcript_id)`


In [15]:
rand_mlu_sumstats.to_csv("../data/rand_mlu_sumstats.csv")

----
#### MLUw plot

Simple plot

In [16]:
%%R -i rand_mlu_sumstats

library('ggplot2')
library('repr')
options(repr.plot.width=6, repr.plot.height=12)

xlabs <- c("C", "NC")

# ara_label <- data.frame(means=c(0),contingent = c(1.5),language="ara") # no adult speech transcribed
deu_label <- data.frame(means=c(5.8),contingent = c(1.5),Language_name="German")
eng_label <- data.frame(means=c(5.8),contingent = c(1.5),Language_name="English")
est_label <- data.frame(means=c(5.8),contingent = c(1.5),Language_name="Estonian")
# fas_label <- data.frame(means=c(5.8),contingent = c(1.5),Language_name="Persian")
fas_ns_label <- data.frame(means=c(6),contingent = c(1.5),Language_name="Persian")
fra_label <- data.frame(means=c(5.8),contingent = c(1.5),Language_name="French")
hrv_label <- data.frame(means=c(5.8),contingent = c(1.5),Language_name="Croatian")
jpn_label <- data.frame(means=c(5.8),contingent = c(1.5),Language_name="Japanese")
kor_label <- data.frame(means=c(5.8),contingent = c(1.5),Language_name="Korean")
# nor_label <- data.frame(means=c(5.8),contingent = c(1.5),Language_name="Norwegian")
nor_ns_label <- data.frame(means=c(5.8),contingent = c(1.5),Language_name="Norwegian")
pol_label <- data.frame(means=c(6),contingent = c(1.5),Language_name="Polish")
por_label <- data.frame(means=c(5.8),contingent = c(1.5),Language_name="Portuguese")
spa_label <- data.frame(means=c(5.8),contingent = c(1.5),Language_name="Spanish")
swe_label <- data.frame(means=c(5.8),contingent = c(1.5),Language_name="Swedish")
# zho_label <- data.frame(means=c(5.8),contingent = c(1.5),Language_name="Mandarin")
zho_ns_label <- data.frame(means=c(6),contingent = c(1.5),Language_name="Mandarin")


p <- ggplot(rand_mlu_sumstats, aes(x = contingent, y = means, color = Language_name)) +
     stat_summary(fun.y=mean, geom="point", shape=19, size=1.75) + 
     stat_summary(fun.data = mean_se, geom = "errorbar", size=1.25, width = .5) +
     facet_wrap(. ~ Language_name,ncol = 7) + 
     geom_text(data = deu_label,label = "***",size=8,color="black") + 
     geom_text(data = eng_label,label = "***",size=8,color="black") +  
     geom_text(data = est_label,label = "**",size=8,color="black") +  
     geom_text(data = fas_ns_label,label = "ns",size=4,color="black",fontface = "italic") +
#      geom_text(data = fas_label,label = "*",size=8, color="black") +
     geom_text(data = fra_label,label = "***",size=8,color="black") +  
     geom_text(data = hrv_label,label = "***",size=8,color="black") + 
     geom_text(data = jpn_label,label = "***",size=8,color="black") + 
     geom_text(data = kor_label,label = "***",size=8,color="black") +  
     geom_text(data = nor_ns_label,label = "**",size=8,color="black") +  
     geom_text(data = pol_label,label = "ns", size=4,color="black",fontface = "italic") +  
     geom_text(data = por_label,label = "***",size=8,color="black") +  
     geom_text(data = spa_label,label = "***",size=8,color="black") + 
     geom_text(data = swe_label,label = "***",size=8,color="black") + 
     geom_text(data = zho_ns_label,label = "ns",size=4,color="black",fontface = "italic") +
#      geom_text(data = zho_label,label = "*",size=8, color="black") +
     ylim(0, 6) +
     labs(tag="B",
          y = "Mean Length of Utterances in Words",
          x = "") +
     theme_classic() +
     scale_x_discrete(labels= xlabs) +
     theme(text = element_text(size=16),
           axis.text.x = element_text(vjust = 0.5, hjust=0.5),
           legend.title = element_blank(),
           legend.background = element_rect(fill=alpha("white",0.90),
                                                            size=0, linetype="dotted",
                                                            colour = "white"),
           legend.text=element_text(size=16))
      ggsave("../figures/token_mlu_rand.pdf", width = 11.7, height = 6.2)

In addition: Warning messages:
1: The `fun.y` argument of `stat_summary()` is deprecated as of ggplot2 3.3.0.
ℹ Please use the `fun` argument instead.
This warning is displayed once every 8 hours.
Call `lifecycle::last_lifecycle_warnings()` to see where this warning was
generated. 
2: Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.
This warning is displayed once every 8 hours.
Call `lifecycle::last_lifecycle_warnings()` to see where this warning was
generated. 
3: The `size` argument of `element_rect()` is deprecated as of ggplot2 3.4.0.
ℹ Please use the `linewidth` argument instead.
This warning is displayed once every 8 hours.
Call `lifecycle::last_lifecycle_warnings()` to see where this warning was
generated. 
4: Removed 173 rows containing non-finite values (`stat_summary()`). 
5: Removed 173 rows containing non-finite values (`stat_summary()`). 


for manuscript

In [17]:
%%R -i rand_mlu_sumstats

library('ggplot2')
library('repr')
options(repr.plot.width=6, repr.plot.height=12)

xlabs <- c("C", "NC")

# ara_label <- data.frame(means=c(0),contingent = c(1.5),language="ara") # no adult speech transcribed
deu_label <- data.frame(means=c(5.65),contingent = c(1.5),Language_name="German")
eng_label <- data.frame(means=c(5.65),contingent = c(1.5),Language_name="English")
est_label <- data.frame(means=c(5.65),contingent = c(1.5),Language_name="Estonian")
# fas_label <- data.frame(means=c(5.8),contingent = c(1.5),Language_name="Persian")
fas_ns_label <- data.frame(means=c(6),contingent = c(1.5),Language_name="Persian")
fra_label <- data.frame(means=c(5.65),contingent = c(1.5),Language_name="French")
hrv_label <- data.frame(means=c(5.65),contingent = c(1.5),Language_name="Croatian")
jpn_label <- data.frame(means=c(5.65),contingent = c(1.5),Language_name="Japanese")
kor_label <- data.frame(means=c(5.65),contingent = c(1.5),Language_name="Korean")
# nor_label <- data.frame(means=c(5.8),contingent = c(1.5),Language_name="Norwegian")
nor_ns_label <- data.frame(means=c(5.65),contingent = c(1.5),Language_name="Norwegian")
pol_label <- data.frame(means=c(6),contingent = c(1.5),Language_name="Polish")
por_label <- data.frame(means=c(5.65),contingent = c(1.5),Language_name="Portuguese")
spa_label <- data.frame(means=c(5.65),contingent = c(1.5),Language_name="Spanish")
swe_label <- data.frame(means=c(5.65),contingent = c(1.5),Language_name="Swedish")
# zho_label <- data.frame(means=c(5.8),contingent = c(1.5),Language_name="Mandarin")
zho_ns_label <- data.frame(means=c(6),contingent = c(1.5),Language_name="Mandarin")


p <- ggplot(rand_mlu_sumstats, aes(x = contingent, y = means, color = Language_name)) +
     stat_summary(fun.y=mean, geom="point", shape=19, size=1.75) + 
     stat_summary(fun.data = mean_se, geom = "errorbar", size=1.25, width = .4) +
     facet_wrap(. ~ Language_name,ncol = 7) + 
     geom_text(data = deu_label,label = "***",size=6,color="black") + 
     geom_text(data = eng_label,label = "***",size=6,color="black") +  
     geom_text(data = est_label,label = "**",size=6,color="black") +  
     geom_text(data = fas_ns_label,label = "ns",size=3,color="black",fontface = "italic") +
#      geom_text(data = fas_label,label = "*",size=8, color="black") +
     geom_text(data = fra_label,label = "***",size=6,color="black") +  
     geom_text(data = hrv_label,label = "***",size=6,color="black") + 
     geom_text(data = jpn_label,label = "***",size=6,color="black") + 
     geom_text(data = kor_label,label = "***",size=6,color="black") +  
     geom_text(data = nor_ns_label,label = "**",size=6,color="black") +  
#      geom_text(data = pol_label,label = "ns", size=3,color="black",fontface = "italic") +  
     geom_text(data = por_label,label = "***",size=6,color="black") +  
     geom_text(data = spa_label,label = "***",size=6,color="black") + 
     geom_text(data = swe_label,label = "***",size=6,color="black") + 
     geom_text(data = zho_ns_label,label = "ns",size=3,color="black",fontface = "italic") +
#      geom_text(data = zho_label,label = "*",size=8, color="black") +
     ylim(0, 6) +
     labs(tag="B",
          y = "Mean Length of Utterances in Words", x = "") +
     theme_classic() +
     scale_x_discrete(labels= xlabs) +
     theme(text = element_text(size=11.5),
           axis.text.x = element_text(vjust = 0.5, hjust=0.5),
           legend.position="none")
      ggsave("../figures/figure_2_B.pdf", width = 11.5, height = 4.2)

In addition: Warning messages:
1: Removed 173 rows containing non-finite values (`stat_summary()`). 
2: Removed 173 rows containing non-finite values (`stat_summary()`). 


Plot + effect estimates

In [18]:
%%R

deu_est_label <- data.frame(means=c(.25),contingent = c(1),Language_name="German")
eng_est_label <- data.frame(means=c(.25),contingent = c(1),Language_name="English")
est_est_label <- data.frame(means=c(.25),contingent = c(1),Language_name="Estonian")
fra_est_label <- data.frame(means=c(.25),contingent = c(1),Language_name="French")
hrv_est_label <- data.frame(means=c(.25),contingent = c(1),Language_name="Croatian")
jpn_est_label <- data.frame(means=c(.25),contingent = c(1),Language_name="Japanese")
kor_est_label <- data.frame(means=c(.25),contingent = c(1),Language_name="Korean")
nor_est_label <- data.frame(means=c(.25),contingent = c(1),Language_name="Norwegian")
por_est_label <- data.frame(means=c(.25),contingent = c(1),Language_name="Portuguese")
spa_est_label <- data.frame(means=c(.25),contingent = c(1),Language_name="Spanish")
swe_est_label <- data.frame(means=c(.25),contingent = c(1),Language_name="Swedish")

p <- p + geom_text(data = deu_est_label,label = "est=-.99",size=4,color="black") +
         geom_text(data = eng_est_label,label = "est=-.62",size=4,color="black") +
         geom_text(data = est_est_label,label = "est=-.48",size=4,color="black") +
         geom_text(data = fra_est_label,label = "est=-.39",size=4,color="black") +
         geom_text(data = hrv_est_label,label = "est=-.46",size=4,color="black") +
         geom_text(data = jpn_est_label,label = "est=-.65",size=4,color="black") +
         geom_text(data = kor_est_label,label = "est=-.47",size=4,color="black") +
#          geom_text(data = nor_est_label,label = "est=-.68",size=4,color="black") +
         geom_text(data = por_est_label,label = "est=-.66",size=4,color="black") +
         geom_text(data = spa_est_label,label = "est=-.42",size=4,color="black") +
         geom_text(data = swe_est_label,label = "est=-.67",size=4,color="black")
         

ggsave("../figures/token_mlu_rand_eff.pdf", width = 11.7, height = 6.2)

In addition: Warning messages:
1: Removed 173 rows containing non-finite values (`stat_summary()`). 
2: Removed 173 rows containing non-finite values (`stat_summary()`). 


\+ sample size

In [19]:
%%R

deu_n_label <- data.frame(means=c(.25),contingent = c(1.7),Language_name="German")
eng_n_label <- data.frame(means=c(.25),contingent = c(1.7),Language_name="English")
est_n_label <- data.frame(means=c(.25),contingent = c(1.7),Language_name="Estonian")
fas_n_label <- data.frame(means=c(.25),contingent = c(1.7),Language_name="Persian")
fra_n_label <- data.frame(means=c(.25),contingent = c(1.7),Language_name="French")
hrv_n_label <- data.frame(means=c(.25),contingent = c(1.7),Language_name="Croatian")
jpn_n_label <- data.frame(means=c(.25),contingent = c(1.7),Language_name="Japanese")
kor_n_label <- data.frame(means=c(.25),contingent = c(1.7),Language_name="Korean")
nor_n_label <- data.frame(means=c(.25),contingent = c(1.7),Language_name="Norwegian")
pol_n_label <- data.frame(means=c(.25),contingent = c(1.7),Language_name="Polish")
por_n_label <- data.frame(means=c(.25),contingent = c(1.7),Language_name="Portuguese")
spa_n_label <- data.frame(means=c(.25),contingent = c(1.7),Language_name="Spanish")
swe_n_label <- data.frame(means=c(.25),contingent = c(1.7),Language_name="Swedish")
zho_n_label <- data.frame(means=c(.25),contingent = c(1.7),Language_name="Mandarin")

deu_sz_label <- data.frame(means=c(.25),contingent = c(2.1),Language_name="German")
eng_sz_label <- data.frame(means=c(.25),contingent = c(2.1),Language_name="English")
est_sz_label <- data.frame(means=c(.25),contingent = c(2.1),Language_name="Estonian")
fas_sz_label <- data.frame(means=c(.25),contingent = c(2.1),Language_name="Persian")
fra_sz_label <- data.frame(means=c(.25),contingent = c(2.1),Language_name="French")
hrv_sz_label <- data.frame(means=c(.25),contingent = c(2.1),Language_name="Croatian")
jpn_sz_label <- data.frame(means=c(.25),contingent = c(2.1),Language_name="Japanese")
kor_sz_label <- data.frame(means=c(.25),contingent = c(2.1),Language_name="Korean")
nor_sz_label <- data.frame(means=c(.25),contingent = c(2.1),Language_name="Norwegian")
pol_sz_label <- data.frame(means=c(.25),contingent = c(2.1),Language_name="Polish")
por_sz_label <- data.frame(means=c(.25),contingent = c(2.1),Language_name="Portuguese")
spa_sz_label <- data.frame(means=c(.25),contingent = c(2.1),Language_name="Spanish")
swe_sz_label <- data.frame(means=c(.25),contingent = c(2.1),Language_name="Swedish")
zho_sz_label <- data.frame(means=c(.25),contingent = c(2.1),Language_name="Mandarin")

p <- p + geom_text(data = deu_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = eng_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = est_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = fas_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = fra_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = hrv_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = jpn_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = kor_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = nor_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = pol_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = por_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = spa_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = swe_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = zho_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = deu_sz_label,label = " = 39",size=4,color="black") +
         geom_text(data = eng_sz_label,label = " = 1005",size=4,color="black") +
         geom_text(data = est_sz_label,label = " = 22",size=4,color="black") +
         geom_text(data = fas_sz_label,label = " = 12",size=4,color="black") +
         geom_text(data = fra_sz_label,label = " = 303",size=4,color="black") +
         geom_text(data = hrv_sz_label,label = " = 79",size=4,color="black") +
         geom_text(data = jpn_sz_label,label = " = 139",size=4,color="black") +
         geom_text(data = kor_sz_label,label = " = 37",size=4,color="black") +
         geom_text(data = nor_sz_label,label = " = 56",size=4,color="black") +
         geom_text(data = pol_sz_label,label = " = 1",size=4,color="black") +
         geom_text(data = por_sz_label,label = " = 24",size=4,color="black") +
         geom_text(data = spa_sz_label,label = " = 31",size=4,color="black") +
         geom_text(data = swe_sz_label,label = " = 16",size=4,color="black") +
         geom_text(data = zho_sz_label,label = " = 2",size=4,color="black")
         

ggsave("../figures/token_mlu_rand_eff_n.pdf", width = 11.7, height = 6.2)

In addition: Warning messages:
1: Removed 173 rows containing non-finite values (`stat_summary()`). 
2: Removed 173 rows containing non-finite values (`stat_summary()`). 


----
#### Statistical analyses

By language

In [24]:
MLUw_dat = rand_dat_inc_cg[['Language_name','num_tokens','contingent','transcript_id','target_child_id','caregiver_type']]

In [30]:
%%R -i MLUw_dat

# vectors for rows to remove from lmer
case_study <- c("Mandarin", "Persian") # only 1 target child analyzed

case_study_cgtype_compare <- c("Korean") # only 1 target child analyzed, varies in CG type

no_cgtype_compare <- c("Portuguese") # only `Mother only`

single_tran <- c("Polish") # only 1 transcript

# nests of models
mlu_nest1 <- MLUw_dat %>%
    filter(!Language_name %in% case_study) %>%
    filter(!Language_name %in% single_tran) %>%
    filter(!Language_name %in% no_cgtype_compare) %>%
    filter(!Language_name %in% case_study_cgtype_compare) %>%
    group_by(Language_name) %>%
    nest() %>%
    mutate(fit = map(data, ~ lmer(num_tokens ~ contingent + caregiver_type +
                                (1|target_child_id) +
                                (1|transcript_id),
                                data = .,
                                REML= FALSE)),
           summary = map(fit, ~ emmeans(., "contingent")),
           contrasts = map(summary, ~ summary(contrast(., method = "pairwise"))),
           effect_size = map2(summary, fit, ~ eff_size(.x, sigma = sigma(.y), edf = df.residual(.y)))) %>%
    select(Language_name, contrasts, effect_size) %>%
    unnest(cols = c(contrasts)) %>%
    mutate(effect_size = map(effect_size, ~ summary(.))) %>%
    unnest() %>%
    mutate(statistic = coalesce(`z.ratio`,`t.ratio`), .before = p.value) %>%
    select (-c(`z.ratio`,`t.ratio`))

mlu_nest2 <- MLUw_dat %>%
    filter(Language_name %in% case_study_cgtype_compare) %>%
    group_by(Language_name) %>%
    nest() %>%
    mutate(fit = map(data, ~ lmer(num_tokens ~ contingent + caregiver_type +
                                (1|transcript_id),
                                data = .,
                                REML= FALSE)),
           summary = map(fit, ~ emmeans(., "contingent")),
           contrasts = map(summary, ~ summary(contrast(., method = "pairwise"))),
           effect_size = map2(summary, fit, ~ eff_size(.x, sigma = sigma(.y), edf = df.residual(.y)))) %>%
    select(Language_name, contrasts, effect_size) %>%
    unnest(cols = c(contrasts)) %>%
    mutate(effect_size = map(effect_size, ~ summary(.))) %>%
    unnest() %>%
    mutate(statistic = coalesce(`z.ratio`), .before = p.value) %>%
    select (-c(`z.ratio`))

mlu_nest3 <- MLUw_dat %>%
    filter(Language_name %in% no_cgtype_compare) %>%
    group_by(Language_name) %>%
    nest() %>%
    mutate(fit = map(data, ~ lmer(num_tokens ~ contingent + 
                                (1|target_child_id) +
                                (1|transcript_id),
                                data = .,
                                REML= FALSE)),
           summary = map(fit, ~ emmeans(., "contingent")),
           contrasts = map(summary, ~ summary(contrast(., method = "pairwise"))),
           effect_size = map2(summary, fit, ~ eff_size(.x, sigma = sigma(.y), edf = df.residual(.y)))) %>%
    select(Language_name, contrasts, effect_size) %>%
    unnest(cols = c(contrasts)) %>%
    mutate(effect_size = map(effect_size, ~ summary(.))) %>%
    unnest() %>%
    mutate(statistic = coalesce(`t.ratio`), .before = p.value) %>%
    select (-c(`t.ratio`))

mlu_nest4 <- MLUw_dat %>%
    filter(Language_name %in% case_study) %>%
    group_by(Language_name) %>%
    nest() %>%
    mutate(fit = map(data, ~ lmer(num_tokens ~ contingent +
                                (1|transcript_id),
                                data = .,
                                REML= FALSE)),
           summary = map(fit, ~ emmeans(., "contingent")),
           contrasts = map(summary, ~ summary(contrast(., method = "pairwise"))),
           effect_size = map2(summary, fit, ~ eff_size(.x, sigma = sigma(.y), edf = df.residual(.y)))) %>%
    select(Language_name, contrasts, effect_size) %>%
    unnest(cols = c(contrasts)) %>%
    mutate(effect_size = map(effect_size, ~ summary(.))) %>%
    unnest() %>%
    mutate(statistic = coalesce(`t.ratio`), .before = p.value) %>%
    select (-c(`t.ratio`))

mlu_nest5 <- MLUw_dat %>%
    filter(Language_name %in% single_tran) %>%
    group_by(Language_name) %>%
    nest() %>%
    mutate(fit = map(data, ~ lm(num_tokens ~ contingent,
                                data = .,
                                REML= FALSE)),
           summary = map(fit, ~ emmeans(., "contingent")),
           contrasts = map(summary, ~ summary(contrast(., method = "pairwise"))),
           effect_size = map2(summary, fit, ~ eff_size(.x, sigma = sigma(.y), edf = df.residual(.y)))) %>%
    select(Language_name, contrasts, effect_size) %>%
    unnest(cols = c(contrasts)) %>%
    mutate(effect_size = map(effect_size, ~ summary(.))) %>%
    unnest() %>%
    rename(statistic = `t.ratio`)
    
# number of transcripts per language
sample_size <- MLUw_dat %>%
    group_by(Language_name) %>%
    summarize(n = n_distinct(transcript_id))
    
# combine lmer summaries and correct p-values for multiple comparisons
emms_all <- list(mlu_nest1, mlu_nest2, mlu_nest3, mlu_nest4, mlu_nest5) %>% 
    reduce(bind_rows) %>%
    mutate(p.value = p.adjust(p.value, "holm", 14)) %>%
    left_join(sample_size)

Joining with `by = join_by(Language_name)`


boundary (singular) fit: see help('isSingular')
boundary (singular) fit: see help('isSingular')
Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'pbkrtest.limit = 5162' (or larger)
[or, globally, 'set emm_options(pbkrtest.limit = 5162)' or larger];
but be warned that this may result in large computation time and memory use.
Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'lmerTest.limit = 5162' (or larger)
[or, globally, 'set emm_options(lmerTest.limit = 5162)' or larger];
but be warned that this may result in large computation time and memory use.
Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'pbkrtest.limit = 90009' (or larger)
[or, globally, 'set emm_options(pbkrtest.limit = 90009)' or larger];
but be warned that this may result in la

format statistics table

In [32]:
%%R

table_maker = function(data) { data %>%
    select(Language_name, n, estimate, SE, statistic, effect.size, p.value) %>%
    `colnames<-`(c("Language", "n", "Estimate", "SE", "Test statistic", "Effect size", "Adjusted p-value")) %>%
    mutate_at(vars(-c(`Adjusted p-value`,Language)), round,2) %>%
    mutate(`Adjusted p-value` = format(round(`Adjusted p-value`,4),nsmall=4)) %>%
    mutate(`Adjusted p-value` = gsub("0.0000","<.0001",`Adjusted p-value`)) %>%
    unite("Estimate (SE)", c('Estimate','SE'), sep=" (") %>%
    mutate(`Estimate (SE)` = paste0(`Estimate (SE)`,")")) %>%
    unite("Language (n)", c('Language','n'), sep=" (") %>%
    mutate(`Language (n)` = paste0(`Language (n)`,")")) %>%
    arrange(`Language (n)`)
    }

MLU_stats_table <- table_maker(emms_all)

kable(MLU_stats_table,"pipe")



|Language (n)    |Estimate (SE) | Test statistic| Effect size|Adjusted p-value |
|:---------------|:-------------|--------------:|-----------:|:----------------|
|Croatian (58)   |-0.36 (0.08)  |          -4.35|       -0.13|0.0002           |
|English (872)   |-0.66 (0.03)  |         -23.20|       -0.22|<.0001           |
|Estonian (22)   |-0.49 (0.14)  |          -3.54|       -0.18|0.0057           |
|French (275)    |-0.53 (0.06)  |          -9.13|       -0.16|<.0001           |
|German (38)     |-0.84 (0.11)  |          -7.67|       -0.25|<.0001           |
|Japanese (160)  |-0.63 (0.03)  |         -21.58|       -0.30|<.0001           |
|Korean (28)     |-0.43 (0.08)  |          -5.52|       -0.21|<.0001           |
|Mandarin (2)    |-0.79 (0.49)  |          -1.62|       -0.25|1.0000           |
|Norwegian (26)  |-0.47 (0.15)  |          -3.16|       -0.17|0.0226           |
|Persian (11)    |-0.48 (0.33)  |          -1.46|       -0.19|1.0000           |
|Polish (1)      |0 (0.7) 

In [15]:
%%R 

# add columns sample and measure and save

MLU_stats_table %>%
    mutate(sample = "rand",
           measure = "mlu") %>%
    write.csv(file = "../data/rand_mlu_stats.csv") 

By play context

In [16]:
rand_mlu_stats_contex = (rand_dat_inc_cg.groupby(["context","target_child_id","transcript_id","contingent"])
                                  .num_tokens
                                  .agg(["mean"])
                                  .reset_index())

rand_mlu_stats_contex =  rand_mlu_stats_contex.rename({'mean': 'means'}, axis=1)

In [17]:
%%R -i rand_mlu_stats_contex

# vectors for rows to remove from lmer
single_tran <- c("Home: interview/unstructured") # only 1 transcript

contex_sample_size <- rand_mlu_stats_contex %>%
    group_by(context) %>%
    summarize(n = n_distinct(transcript_id))

mlu_contex_nest_1 <- rand_mlu_stats_contex %>%
    filter(!context %in% single_tran) %>%
    group_by(context) %>%
    nest() %>%
    mutate(fit = map(data, ~ lmer(means ~ contingent +
                                (1|transcript_id),
                                data = .,
                                REML= FALSE)),
           summary = map(fit, ~ emmeans(., "contingent")),
           contrasts = map(summary, ~ summary(contrast(., method = "pairwise"))),
           effect_size = map2(summary, fit, ~ eff_size(.x, sigma = sigma(.y), edf = df.residual(.y)))) %>%
    select(context, contrasts, effect_size) %>%
    unnest(cols = c(contrasts)) %>%
    mutate(effect_size = map(effect_size, ~ summary(.))) %>%
    unnest() %>%
    mutate(statistic = coalesce(`t.ratio`), .before = p.value) %>%
    select (-c(`t.ratio`))
    
mlu_contex_nest_2 <- rand_mlu_stats_contex %>%
    filter(context %in% single_tran) %>%
    group_by(context) %>%
    nest() %>%
    mutate(fit = map(data, ~ lm(means ~ contingent,
                                data = .,
                                REML= FALSE)),
           summary = map(fit, ~ emmeans(., "contingent")),
           contrasts = map(summary, ~ summary(contrast(., method = "pairwise")))) %>%
    select(context, contrasts) %>%
    unnest(cols = c(contrasts))  %>%
    rename(statistic = `t.ratio`)

# combine lmer summaries and correct p-values for multiple comparisons
context_emms_all <- list(mlu_contex_nest_1, mlu_contex_nest_2) %>% 
    reduce(bind_rows) %>%
    mutate(p.value = p.adjust(p.value, "holm", 7)) %>%
    left_join(contex_sample_size)

Joining with `by = join_by(context)`


In [18]:
%%R

table_maker = function(data) { data %>%
    select(context, n, estimate, SE, statistic, effect.size, p.value) %>%
    `colnames<-`(c("Play context", "n", "Estimate", "SE", "Test statistic", "Effect size", "Adjusted p-value")) %>%
    mutate_at(vars(-c(`Adjusted p-value`,`Play context`)), round,2) %>%
    mutate(`Adjusted p-value` = format(round(`Adjusted p-value`,4),nsmall=4)) %>%
    mutate(`Adjusted p-value` = gsub("0.0000","<.0001",`Adjusted p-value`)) %>%
    unite("Estimate (SE)", c('Estimate','SE'), sep=" (") %>%
    mutate(`Estimate (SE)` = paste0(`Estimate (SE)`,")")) %>%
    unite("Play context (n)", c(`Play context`,'n'), sep=" (") %>%
    mutate(`Play context (n)` = paste0(`Play context (n)`,")")) %>%
    arrange(`Play context (n)`)
    }
    
mlu_context_stats_table <- table_maker(context_emms_all)

kable(mlu_context_stats_table)



|Play context (n)                  |Estimate (SE) | Test statistic| Effect size|Adjusted p-value |
|:---------------------------------|:-------------|--------------:|-----------:|:----------------|
|Home: book reading (28)           |-0.47 (0.08)  |          -5.64|       -1.54|<.0001           |
|Home: interview/unstructured (1)  |0 (NaN)       |            NaN|          NA|NaN              |
|Home: other (20)                  |-0.51 (0.13)  |          -3.85|       -1.25|0.0066           |
|Home: unstructured (898)          |-0.65 (0.04)  |         -16.92|       -0.80|<.0001           |
|Lab: interview/unstructured (360) |-0.44 (0.1)   |          -4.57|       -0.35|<.0001           |
|Lab: unstructured (26)            |-0.1 (0.1)    |          -0.96|       -0.27|1.0000           |
|Other: unstructured (128)         |-0.8 (0.07)   |         -12.10|       -1.52|<.0001           |
|Unreported (101)                  |-0.38 (0.09)  |          -4.38|       -0.62|0.0002           |


By context, dropping English

In [21]:
# drop English
rand_dat_inc_cg_no_eng = rand_dat_inc_cg[rand_dat_inc_cg["Language_name"] != "English"]

rand_mlu_stats_contex_no_eng = (rand_dat_inc_cg_no_eng.groupby(["context","target_child_id","transcript_id","contingent"])
                                  .num_tokens
                                  .agg(["mean"])
                                  .reset_index())

rand_mlu_stats_contex_no_eng =  rand_mlu_stats_contex_no_eng.rename({'mean': 'means'}, axis=1)

In [22]:
%%R -i rand_mlu_stats_contex_no_eng

# # vectors for rows to remove from lmer
single_tran <- c("Home: interview/unstructured") # only 1 transcript

contex_sample_size_no_eng <- rand_mlu_stats_contex_no_eng %>%
    group_by(context) %>%
    summarize(n = n_distinct(transcript_id))

mlu_contex_nest_1_no_eng <- rand_mlu_stats_contex_no_eng %>%
    filter(!context %in% single_tran) %>%
    group_by(context) %>%
    nest() %>%
    mutate(fit = map(data, ~ lmer(means ~ contingent +
                                (1|transcript_id),
                                data = .,
                                REML= FALSE)),
           summary = map(fit, ~ emmeans(., "contingent")),
           contrasts = map(summary, ~ summary(contrast(., method = "pairwise"))),
           effect_size = map2(summary, fit, ~ eff_size(.x, sigma = sigma(.y), edf = df.residual(.y)))) %>%
    select(context, contrasts, effect_size) %>%
    unnest(cols = c(contrasts)) %>%
    mutate(effect_size = map(effect_size, ~ summary(.))) %>%
    unnest() %>%
    mutate(statistic = coalesce(`t.ratio`), .before = p.value) %>%
    select (-c(`t.ratio`))

mlu_contex_nest_2_no_eng <- rand_mlu_stats_contex_no_eng %>%
    filter(context %in% single_tran) %>%
    group_by(context) %>%
    nest() %>%
    mutate(fit = map(data, ~ lm(means ~ contingent,
                                data = .,
                                REML= FALSE)),
           summary = map(fit, ~ emmeans(., "contingent")),
           contrasts = map(summary, ~ summary(contrast(., method = "pairwise")))) %>%
    select(context, contrasts) %>%
    unnest(cols = c(contrasts))  %>%
    rename(statistic = `t.ratio`)
    
# combine lmer summaries and correct p-values for multiple comparisons
context_no_eng_emms_all <- list(mlu_contex_nest_1_no_eng, mlu_contex_nest_2_no_eng) %>% 
    reduce(bind_rows) %>%
    mutate(p.value = p.adjust(p.value, "holm", 3)) %>%
    left_join(contex_sample_size_no_eng)

Joining with `by = join_by(context)`


In [23]:
%%R 

table_maker = function(data) { data %>%
    select(context, n, estimate, SE, statistic, effect.size, p.value) %>%
    `colnames<-`(c("Play context", "n", "Estimate", "SE", "Test statistic", "Effect size", "Adjusted p-value")) %>%
    mutate_at(vars(-c(`Adjusted p-value`,`Play context`)), round,2) %>%
    mutate(`Adjusted p-value` = format(round(`Adjusted p-value`,4),nsmall=4)) %>%
    mutate(`Adjusted p-value` = gsub("0.0000","<.0001",`Adjusted p-value`)) %>%
    unite("Estimate (SE)", c('Estimate','SE'), sep=" (") %>%
    mutate(`Estimate (SE)` = paste0(`Estimate (SE)`,")")) %>%
    unite("Play context (n)", c(`Play context`,'n'), sep=" (") %>%
    mutate(`Play context (n)` = paste0(`Play context (n)`,")")) %>%
    arrange(`Play context (n)`)
    }
    
mlu_context_stats_table_no_eng <- table_maker(context_no_eng_emms_all)

kable(mlu_context_stats_table_no_eng)



|Play context (n)                 |Estimate (SE) | Test statistic| Effect size|Adjusted p-value |
|:--------------------------------|:-------------|--------------:|-----------:|:----------------|
|Home: book reading (28)          |-0.47 (0.08)  |          -5.64|       -1.54|<.0001           |
|Home: interview/unstructured (1) |0 (NaN)       |            NaN|          NA|NaN              |
|Home: unstructured (560)         |-0.54 (0.04)  |         -12.68|       -0.76|<.0001           |


By language family

In [11]:
rand_mlu_stats_fam = (rand_dat_inc_cg.groupby(["Language_Family","target_child_id","transcript_id","contingent"])
                                  .num_tokens
                                  .agg(["mean"])
                                  .reset_index())
rand_mlu_stats_fam =  rand_mlu_stats_fam.rename({'mean': 'means'}, axis=1)

In [12]:
%%R -i rand_mlu_stats_fam

# plot

library('ggplot2')

p <- ggplot(rand_mlu_stats_fam, aes(x = contingent, y = means, color = Language_Family)) +
     stat_summary(fun.y=mean, geom="point", shape=19, size=1.75) + 
     stat_summary(fun.data = mean_se, geom = "errorbar", size=1.25, width = .5) +
     facet_wrap(. ~ Language_Family,ncol=5) + 
     labs(y = "Mean length of utterance", x = "") +
     theme_classic() +
     theme(text = element_text(size=16),
           axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1),
           legend.title = element_blank(),
           legend.background = element_rect(fill=alpha("white",0.90),
                                                            size=0, linetype="dotted",
                                                            colour = "white"),
           legend.text=element_text(size=16))
    ggsave("../figures/token_mlu_family.pdf", width = 11.7, height = 6.2)
    
# statistical analysis

fam_sample_size <- rand_mlu_stats_fam %>%
    group_by(Language_Family) %>%
    summarize(n = n_distinct(transcript_id))
    
mlu_fam_nest <- rand_mlu_stats_fam %>%
    group_by(Language_Family) %>%
    nest() %>%
    mutate(fit = map(data, ~ lmer(means ~ contingent +
                                (1|transcript_id),
                                data = .,
                                REML= FALSE)),
           summary = map(fit, ~ emmeans(., "contingent")),
           contrasts = map(summary, ~ summary(contrast(., method = "pairwise"))),
           effect_size = map2(summary, fit, ~ eff_size(.x, sigma = sigma(.y), edf = df.residual(.y)))) %>%
    select(Language_Family, contrasts, effect_size) %>%
    unnest(cols = c(contrasts)) %>%
    mutate(effect_size = map(effect_size, ~ summary(.))) %>%
    unnest() %>%
    mutate(statistic = coalesce(`t.ratio`), .before = p.value) %>%
    select (-c(`t.ratio`)) %>%
    mutate(p.value = p.adjust(p.value, "holm", 5)) %>%
    left_join(fam_sample_size)
    
    table_maker = function(data) { data %>%
    select(Language_Family, n, estimate, SE, statistic, effect.size, p.value) %>%
    `colnames<-`(c("Language Family", "n", "Estimate", "SE", "Test statistic", "Effect size", "Adjusted p-value")) %>%
    mutate_at(vars(-c(`Adjusted p-value`,`Language Family`)), round,2) %>%
    mutate(`Adjusted p-value` = format(round(`Adjusted p-value`,4),nsmall=4)) %>%
    mutate(`Adjusted p-value` = gsub("0.0000","<.0001",`Adjusted p-value`)) %>%
    unite("Estimate (SE)", c('Estimate','SE'), sep=" (") %>%
    mutate(`Estimate (SE)` = paste0(`Estimate (SE)`,")")) %>%
    unite("Language Family (n)", c(`Language Family`,'n'), sep=" (") %>%
    mutate(`Language Family (n)` = paste0(`Language Family (n)`,")")) %>%
    arrange(`Language Family (n)`)
    }
    
mlu_fam_stats_table <- table_maker(mlu_fam_nest)

kable(mlu_fam_stats_table)

Joining, by = "Language_Family"


|Language Family (n)  |Estimate (SE) | Test statistic| Effect size|Adjusted p-value |
|:--------------------|:-------------|--------------:|-----------:|:----------------|
|Indo-European (1483) |-0.58 (0.04)  |         -15.22|       -0.57|<.0001           |
|Japonic (160)        |-0.63 (0.04)  |         -17.67|       -1.98|<.0001           |
|Koreanic (28)        |-0.46 (0.07)  |          -6.48|       -1.76|<.0001           |
|Sino-Tibetan (2)     |-0.93 (0.12)  |          -7.66|      -10.83|0.0078           |
|Uralic (22)          |-0.62 (0.11)  |          -5.54|       -1.71|0.0001           |


By language genus

In [13]:
rand_mlu_stats_gen = (rand_dat_inc_cg.groupby(["Language_Genus","target_child_id","transcript_id","contingent"])
                                  .num_tokens
                                  .agg(["mean"])
                                  .reset_index())
rand_mlu_stats_gen = rand_mlu_stats_gen.rename({'mean': 'means'}, axis=1)

In [30]:
%%R -i rand_mlu_stats_gen

library('ggplot2')

# plot

p <- ggplot(rand_mlu_stats_gen, aes(x = contingent, y = means, color = Language_Genus)) +
     stat_summary(fun.y=mean, geom="point", shape=19, size=1.75) + 
     stat_summary(fun.data = mean_se, geom = "errorbar", size=1.25, width = .5) +
     facet_wrap(. ~ Language_Genus,ncol=8) + 
     labs(y = "Mean length of utterance", x = "") +
     theme_classic() +
     theme(text = element_text(size=16),
           axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1),
           legend.title = element_blank(),
           legend.background = element_rect(fill=alpha("white",0.90),
                                                            size=0, linetype="dotted",
                                                            colour = "white"),
           legend.text=element_text(size=16))
    ggsave("../figures/token_mlu_genus.pdf", width = 11.7, height = 6.2)
    
# statistical analysis

gen_sample_size <- rand_mlu_stats_gen %>%
    group_by(Language_Genus) %>%
    summarize(n = n_distinct(transcript_id))
    
mlu_gen_nest <- rand_mlu_stats_gen %>%
    group_by(Language_Genus) %>%
    nest() %>%
    mutate(fit = map(data, ~ lmer(means ~ contingent +
                                (1|transcript_id),
                                data = .,
                                REML= FALSE)),
           summary = map(fit, ~ emmeans(., "contingent")),
           contrasts = map(summary, ~ summary(contrast(., method = "pairwise"))),
           effect_size = map2(summary, fit, ~ eff_size(.x, sigma = sigma(.y), edf = df.residual(.y)))) %>%
    select(Language_Genus, contrasts, effect_size) %>%
    unnest(cols = c(contrasts)) %>%
    mutate(effect_size = map(effect_size, ~ summary(.))) %>%
    unnest() %>%
    mutate(statistic = coalesce(`t.ratio`), .before = p.value) %>%
    select (-c(`t.ratio`)) %>%
    mutate(p.value = p.adjust(p.value, "holm", 5)) %>%
    left_join(gen_sample_size)
    
table_maker = function(data) { data %>%
    select(Language_Genus, n, estimate, SE, statistic, effect.size, p.value) %>%
    `colnames<-`(c("Language Genus", "n", "Estimate", "SE", "Test statistic", "Effect size", "Adjusted p-value")) %>%
    mutate_at(vars(-c(`Adjusted p-value`,`Language Genus`)), round,2) %>%
    mutate(`Adjusted p-value` = format(round(`Adjusted p-value`,4),nsmall=4)) %>%
    mutate(`Adjusted p-value` = gsub("0.0000","<.0001",`Adjusted p-value`)) %>%
    unite("Estimate (SE)", c('Estimate','SE'), sep=" (") %>%
    mutate(`Estimate (SE)` = paste0(`Estimate (SE)`,")")) %>%
    unite("Language Genus (n)", c(`Language Genus`,'n'), sep=" (") %>%
    mutate(`Language Genus (n)` = paste0(`Language Genus (n)`,")")) %>%
    arrange(`Language Genus (n)`)
    }
    
lexdiv_gen_stats_table <- table_maker(mlu_gen_nest)

kable(lexdiv_gen_stats_table)

R[write to console]: boundary (singular) fit: see help('isSingular')

R[write to console]: boundary (singular) fit: see help('isSingular')



Joining, by = "Language_Genus"


|Language Genus (n) |Estimate (SE) | Test statistic| Effect size|Adjusted p-value |
|:------------------|:-------------|--------------:|-----------:|:----------------|
|Chinese (2)        |-0.93 (0.12)  |          -7.66|      -10.83|0.0078           |
|Finnic (22)        |-0.62 (0.11)  |          -5.54|       -1.71|0.0001           |
|Germanic (1077)    |-0.63 (0.05)  |         -12.91|       -0.57|<.0001           |
|Iranian (11)       |-0.44 (0.28)  |          -1.59|       -0.71|0.6855           |
|Japanese (160)     |-0.63 (0.04)  |         -17.67|       -1.98|<.0001           |
|Korean (28)        |-0.46 (0.07)  |          -6.48|       -1.76|<.0001           |
|Romance (335)      |-0.48 (0.06)  |          -7.68|       -0.60|<.0001           |
|Savlic (60)        |-0.34 (0.08)  |          -4.29|       -0.80|0.0003           |


By agglutinative status

In [15]:
rand_mlu_stats_aggl = (rand_dat_inc_cg.groupby(["Agglutinative","target_child_id","transcript_id","contingent"])
                                  .num_tokens
                                  .agg(["mean"])
                                  .reset_index())
rand_mlu_stats_aggl = rand_mlu_stats_aggl.rename({'mean': 'means'}, axis=1)

In [31]:
%%R -i rand_mlu_stats_aggl

library('ggplot2')

# plot

p <- ggplot(rand_mlu_stats_aggl, aes(x = contingent, y = means, color = Agglutinative)) +
     stat_summary(fun.y=mean, geom="point", shape=19, size=1.75) + 
     stat_summary(fun.data = mean_se, geom = "errorbar", size=1.25, width = .5) +
     facet_wrap(. ~ Agglutinative) + 
     labs(y = "Mean length of utterance", x = "") +
     theme_classic() +
     theme(text = element_text(size=16),
           axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1),
           legend.title = element_blank(),
           legend.background = element_rect(fill=alpha("white",0.90),
                                                            size=0, linetype="dotted",
                                                            colour = "white"),
           legend.text=element_text(size=16))
    ggsave("../figures/token_mlu_aggl.pdf", width = 11.7, height = 6.2)
    
# statistical analysis
    
agg_sample_size <- rand_mlu_stats_aggl %>%
    group_by(Agglutinative) %>%
    summarize(n = n_distinct(transcript_id))
    
mlu_agg_nest <- rand_mlu_stats_aggl %>%
    group_by(Agglutinative) %>%
    nest() %>%
    mutate(fit = map(data, ~ lmer(means ~ contingent +
                                (1|transcript_id),
                                data = .,
                                REML= FALSE)),
           summary = map(fit, ~ emmeans(., "contingent")),
           contrasts = map(summary, ~ summary(contrast(., method = "pairwise"))),
           effect_size = map2(summary, fit, ~ eff_size(.x, sigma = sigma(.y), edf = df.residual(.y)))) %>%
    select(Agglutinative, contrasts, effect_size) %>%
    unnest(cols = c(contrasts)) %>%
    mutate(effect_size = map(effect_size, ~ summary(.))) %>%
    unnest() %>%
    mutate(statistic = coalesce(`t.ratio`), .before = p.value) %>%
    select (-c(`t.ratio`)) %>%
    mutate(p.value = p.adjust(p.value, "holm", 5)) %>%
    left_join(agg_sample_size)
    
table_maker = function(data) { data %>%
    select(Agglutinative, n, estimate, SE, statistic, effect.size, p.value) %>%
    `colnames<-`(c("Agglutinative Status", "n", "Estimate", "SE", "Test statistic", "Effect size", "Adjusted p-value")) %>%
    mutate_at(vars(-c(`Adjusted p-value`,`Agglutinative Status`)), round,2) %>%
    mutate(`Adjusted p-value` = format(round(`Adjusted p-value`,4),nsmall=4)) %>%
    mutate(`Adjusted p-value` = gsub("0.0000","<.0001",`Adjusted p-value`)) %>%
    unite("Estimate (SE)", c('Estimate','SE'), sep=" (") %>%
    mutate(`Estimate (SE)` = paste0(`Estimate (SE)`,")")) %>%
    unite("Agglutinative Status (n)", c(`Agglutinative Status`,'n'), sep=" (") %>%
    mutate(`Agglutinative Status (n)` = paste0(`Agglutinative Status (n)`,")")) %>%
    arrange(`Agglutinative Status (n)`)
    }
    
mlu_agg_stats_table <- table_maker(mlu_agg_nest)

kable(mlu_agg_stats_table)

Joining, by = "Agglutinative"


|Agglutinative Status (n) |Estimate (SE) | Test statistic| Effect size|Adjusted p-value |
|:------------------------|:-------------|--------------:|-----------:|:----------------|
|0 (1485)                 |-0.58 (0.04)  |         -15.25|       -0.57|<.0001           |
|1 (210)                  |-0.61 (0.03)  |         -19.47|       -1.90|<.0001           |
